# File Information
This file is for the assignment 5 of Monte Carlo Method at University Paris Dauphine master 2 MASEF

Author: Yu Xiang

Contact: shawnxiangyu@yahoo.com

In [20]:
# import libaries and set the setting for plot
%matplotlib inline
import numpy as np
from scipy import linalg   
from scipy.stats import norm
import time
import matplotlib.pyplot as plt
import math
from scipy.stats.stats import pearsonr 
import pandas as pd
from scipy import stats 
norm_ppf = stats.norm.ppf

# choose a large font size by default and use tex for math
fontsize = 10
params = {'axes.labelsize': fontsize + 2,
      'font.size': fontsize + 2,
      'legend.fontsize': fontsize + 2,
      'xtick.labelsize': fontsize,
      'ytick.labelsize': fontsize}
plt.rcParams.update(params)

## Exercise 1: Payoff Regularization


In [21]:
# parameters
r = 0 
sigma = 0.25 
X0 = 100
T = 1 
a = 95
b = 105

ninty5_quantile = norm_ppf(0.975)

A stochastic process $X_t$ is said to follow a GBM if it satisfies the following stochastic differential equation (SDE):

$$ dX_t = \mu X_t\,dt + \sigma X_t\,dW_t $$

For an arbitrary initial value $X_0$ the above SDE has the analytic solution (under Itô calculus|Itô's interpretation):
$$ X_t = X_0\exp\left( \left(\mu - \frac{\sigma^2}{2} \right)t + \sigma W_t\right)$$

####  Confidence interval
According to central limit theorem:  
Let $\{X_1,..., X_n\}$ be a random sample of size $n$ — that is, a sequence of independent and identically distributed (i.i.d.) random variables drawn from a distribution of expected value given by $\mu$ and finite variance given by $\sigma$. Suppose we are interested in the ample mean|sample average: 

$$S_n := \frac{X_1+\cdots+X_n}{n}$$

Then, we have: 
        
$$\sqrt{n}\left(S_n - \mu\right)\ \xrightarrow{d}\ N\left(0,\sigma^2\right)$$   

The population variance $\sigma^2$ could be estimated by the sample variance $\sigma_n^2$

The sample variance is calculated as follows: 
$$s^2 = \frac{n}{n-1} \sigma_y^2 =  \frac{n}{n-1}(\operatorname{E}\left[Y^2 \right] - \operatorname{E}[Y]^2) $$




In [22]:
# Compute the European call  via Monte Carlo Method Based on Black Scholes model
def sim_XT_WT(T=1, nr_steps=500, X0=100, r=0, sigma=0.25, nr_sims=10000): 
    
    dt = T / nr_steps  # time grid: tg[0] = 0,  tg[-1] = T, tg length: n + 1 (t0, t1, ..., tn)
    XT = np.ones((nr_sims, 1)) * X0
    W =  np.zeros((nr_sims, 1))
        
    for i in range(nr_steps): 
        dW = dt ** 0.5 * np.random.randn(nr_sims, 1)
        XT *= np.exp((r - sigma ** 2 / 2) * dt + sigma * dW)  
        W += dW
    return XT, W



def G(Xs, a=95, b=105):
    
    inbetw_id = np.logical_and(Xs >= a, Xs <= b)
    bigger_than_b = Xs > b
    Gx = (np.log(Xs) - np.log(a)) * inbetw_id + (np.log(b) - np.log(a)) * bigger_than_b
    
    return Gx

In [23]:
# 1.1
nr_sims = [10000, 50000, 100000]
nr_cases = len(nr_sims)

value_conf_interval = np.zeros((nr_cases, 5))
rug_value_conf_interval = np.zeros((nr_cases, 5))

for i in range(nr_cases): 
    nr_simi = nr_sims[i]
    Xs, W = sim_XT_WT(nr_sims=nr_simi)
    
    Gx = G(Xs)
    rug_Ep = np.mean(Gx * W / (sigma * T)) 
    
    
    inbetw_id = np.logical_and(Xs >= a, Xs <= b)
    ep = np.sum(inbetw_id) /len(inbetw_id) # expectaction of the probablity
    
    
    var_n = nr_simi /(nr_simi - 1) * (ep - ep ** 2) 
    std_n = var_n ** 0.5
    half_int = 1.96 * std_n / nr_simi ** 0.5 
    conf_interval = np.round([ep - half_int,  ep + half_int],4)
    
    value_conf_interval[i, 0] = nr_simi
    value_conf_interval[i, 1] = ep

    value_conf_interval[i, 2:4] = conf_interval
    value_conf_interval[i, 4] = half_int * 2
    
    
    rug_var = nr_simi /(nr_simi - 1) * (np.mean((Gx * W / (sigma * T)) ** 2)  - rug_Ep ** 2)
    rug_half_int = 1.96 * rug_var ** 0.5 / nr_simi ** 0.5 
    rug_conf_interval = np.round([rug_Ep - rug_half_int,  rug_Ep + rug_half_int],4)
    
    
    rug_value_conf_interval[i, 0] = nr_simi
    rug_value_conf_interval[i, 1] = rug_Ep

    rug_value_conf_interval[i, 2:4] = rug_conf_interval
    rug_value_conf_interval[i, 4] = rug_half_int * 2
 

 
NMC = value_conf_interval
PRMC = rug_value_conf_interval

In [24]:
# normal simulation result
import pandas as pd
result = pd.DataFrame(NMC,
                      columns=['Nr_sims', 'probability', 
                               '95% conf_lower_bound', '95% conf_upper_bound','interval_length'])

result

,Nr_sims,probability,95% conf_lower_bound,95% conf_upper_bound,interval_length
0,10000.0,0.15900,0.1518,0.1662,0.014335
1,50000.0,0.15798,0.1548,0.1612,0.006394
2,100000.0,0.15827,0.1560,0.1605,0.004525


In [25]:
# regularization result

import pandas as pd
result = pd.DataFrame(PRMC,
                      columns=['Nr_sims', 'probability', 
                               '95% conf_lower_bound', '95% conf_upper_bound','interval_length'])



result

,Nr_sims,probability,95% conf_lower_bound,95% conf_upper_bound,interval_length
0,10000.0,0.158103,0.1535,0.1627,0.009275
1,50000.0,0.157143,0.1551,0.1592,0.004113
2,100000.0,0.156821,0.1554,0.1583,0.002906


### Comment: 

As we can see, the regularization methods has smaller variance compared to the normal simulation. 
Both methods's variance decreases when the number of simulation increases, and they seems to converges to a same result. 

In [26]:
# 1.3 Compare the result
abs_p_diff = NMC[:,1] - PRMC[:,1]
abs_conf_inter_diff = NMC[:,4] - PRMC[:,4] 
times_conf_inter_len = NMC[:,4] / PRMC[:,4] 
Compare = np.hstack((NMC, PRMC[:,1:], abs_p_diff[:,np.newaxis],
                     abs_conf_inter_diff[:,np.newaxis], times_conf_inter_len[:,np.newaxis]))


# display summary and compare
import pandas as pd
Exercise_1_result = pd.DataFrame(Compare,
                      columns=['Nr_sim','N_p','N_lower', 'N_upper',
                               'N_intlen', 'PR_p','PR_lower', 'PR_upper',
                               'PR_intlen','diff_p', 'diff_intlen', 'times_intlen'])


Exercise_1_result

,Nr_sim,N_p,N_lower,N_upper,N_intlen,PR_p,PR_lower,PR_upper,PR_intlen,diff_p,diff_intlen,times_intlen
0,10000.0,0.15900,0.1518,0.1662,0.014335,0.158103,0.1535,0.1627,0.009275,0.000897,0.005060,1.545602
1,50000.0,0.15798,0.1548,0.1612,0.006394,0.157143,0.1551,0.1592,0.004113,0.000837,0.002281,1.554585
2,100000.0,0.15827,0.1560,0.1605,0.004525,0.156821,0.1554,0.1583,0.002906,0.001449,0.001618,1.556944


### Comment: 
Basically, the normal Monte Carlo methods's confidence interval length is about 1.55 times that of Payoff Regularization. 

## Exercise 2: Control Variate
### 2.0 pre-computation

In [42]:
def bs_euro_call(St, t, K): 
    rt = T - t  # remaining time to maturity
    d1 = 1/ (sigma * rt ** 0.5) * (np.log(St/K) + (r + sigma ** 2 / 2) * rt)
    d2 = d1 - sigma * rt ** 0.5 
    PV_K = K * np.exp(-r * rt)    
    C_St_t = norm.cdf(d1) * St - norm.cdf(d2) * PV_K
    
    return C_St_t

In [43]:
r = 0
sigma = 0.3
X0 = 100
T = 1

Ks = [80, 150]
Vs = np.zeros(2)
for i in range(len(Ks)): 
    K = Ks[i]
    Vs[i] = bs_euro_call(X0, 0, K)
    
      # print ('The European call with S = ' + str(X0), ' at time t = ' + str(0), 'is equal to: ' + str(C_t0))      
Vs        

array([23.5343901 ,  1.48589383])

In [27]:
# 2.1 simple/ Normal Monte Carlo simulation

r = 0 
sigma = 0.3
X0 = 100
T = 1

Ks = [80, 150]
nr_sims = [10000,  100000]
nr_cases = len(Ks) * len(nr_sims)
value_conf_interval = np.zeros((nr_cases, 6))

i = 0
for ki in range(len(Ks)): 
    
    K = Ks[ki]
    
    for si in range(len(nr_sims)): 
        nr_simi = nr_sims[si]
        
        XT, WT = sim_XT_WT(T=T, X0=X0, r=r, sigma=sigma, nr_sims=nr_simi)
        
        price = np.mean(np.exp(-r * T) * np.maximum(XT - K, 0))
        var_n = nr_simi / (nr_simi - 1) * (np.mean((np.exp(-r * T) \
                        * np.maximum(XT - K,0)) ** 2) - price **2) 
    
        # 95% interval corresponds to 1.96 * std_n / (sqrt(n))
        half_int = 1.96 * var_n ** 0.5 / nr_simi ** 0.5 

        conf_interval = np.round([price - half_int,  price + half_int],4)      
        value_conf_interval[i, 0] = nr_simi
        value_conf_interval[i, 1] = K
        value_conf_interval[i, 2] = price
        
        value_conf_interval[i, 3:5] = conf_interval
        value_conf_interval[i, 5] = half_int * 2
        
        i += 1


NMC = value_conf_interval

In [28]:
# display the result from simple Monte Carlo Method
import pandas as pd
result = pd.DataFrame(NMC,
                      columns=['Nr_sims', 'strike_K', 'price',
                               '95% conf_lower_bound', '95% conf_upper_bound','interval_length'])


result

,Nr_sims,strike_K,price,95% conf_lower_bound,95% conf_upper_bound,interval_length
0,10000.0,80.0,23.421894,22.9072,23.9366,1.029357
1,100000.0,80.0,23.530067,23.3635,23.6966,0.333106
2,10000.0,150.0,1.457856,1.3061,1.6096,0.303420
3,100000.0,150.0,1.447885,1.3994,1.4963,0.096893


In [45]:
# 2.2 Monte Carlo simulation with Control Variate
r = 0 
sigma = 0.3
X0 = 100
T = 1

Ks = [80, 150]
nr_sims = [10000,  100000]
nr_cases = len(Ks) * len(nr_sims)
cv_value_conf_interval = np.zeros((nr_cases, 7))

i = 0

for ki in range(len(Ks)): 
    
    K = Ks[ki]
    
    for si in range(len(nr_sims)): 
        nr_simi = nr_sims[si]
        
        part_nrsim = int(nr_simi / 4.0) # 1/4 simulations to estimate b_star
        
        nr_simforp = nr_simi - part_nrsim
        
        XT, WT = sim_XT_WT(T=T, X0=X0, r=r, sigma=sigma, nr_sims=nr_simi)
        Y = np.exp(-r * T) * np.maximum(XT - K, 0)
        
        XM = XT[:part_nrsim]
        XN = XT[part_nrsim:] 
        XM_mean = np.mean(XM)
        XN_mean = np.mean(XN)
        
        
        YM = Y[:part_nrsim]
        YN = Y[part_nrsim:]
        YM_mean = np.mean(YM)
        YN_mean = np.mean(YN)
        
        
        # half the simulation is used for calculating b * star
        b_star = np.sum((XM-XM_mean) * (YM-YM_mean)) / np.sum((XM - XM_mean) ** 2)  
        
        # remaining half is used to calculate the price and variance
        price = 1 / nr_simforp * np.sum(YN + b_star * (XN - X0))
        
        
        # now we need to calculate the variance 
        VarY =  (nr_simforp )  / (nr_simforp - 1) * (np.mean(YN ** 2) - YN_mean ** 2)
        rho_XY = pearsonr(XN, YN)[0][0]
        Var_YN = VarY * (1 - rho_XY ** 2)
        
        # 95% interval corresponds to 1.96 * std_n / (sqrt(n))
        
        half_int = ninty5_quantile * Var_YN ** 0.5 / nr_simforp ** 0.5

        conf_interval = np.round([price - half_int,  price + half_int],4)      
        cv_value_conf_interval[i, 0] = nr_simforp
        cv_value_conf_interval[i, 1] = K
        cv_value_conf_interval[i, 2] = price
        
        
        cv_value_conf_interval[i, 3:5] = conf_interval
        cv_value_conf_interval[i, 5] = half_int * 2
        
        cv_value_conf_interval[i, 6] = rho_XY
        
        i += 1


CVMC = cv_value_conf_interval

# display the result from simple Monte Carlo Method
import pandas as pd
result = pd.DataFrame(CVMC,
                      columns=['Nr_sims', 'strike_K', 'price',
                               '95% conf_lower_bound', '95% conf_upper_bound','interval_length', 'rho_xy'])


result

,Nr_sims,strike_K,price,95% conf_lower_bound,95% conf_upper_bound,interval_length,rho_xy
0,7500.0,80.0,23.398018,23.2580,23.5381,0.280086,0.973940
1,75000.0,80.0,23.288969,23.2461,23.3318,0.085646,0.974267
2,7500.0,150.0,1.420257,1.2697,1.5708,0.301016,0.572447
3,75000.0,150.0,1.334797,1.2899,1.3797,0.089760,0.561009


In [44]:
print ('real value is : ', Vs)

real value is :  [23.5343901   1.48589383]


In [39]:
# 2.3 Compare the result
abs_p_diff = NMC[:,2] - CVMC[:,2]
abs_conf_inter_diff = NMC[:,5] - CVMC[:,5] 
times_conf_inter_len = NMC[:,5] / CVMC[:,5] 
Compare = np.hstack((NMC, CVMC[:,2:], abs_p_diff[:,np.newaxis],
                     abs_conf_inter_diff[:,np.newaxis], times_conf_inter_len[:,np.newaxis]))


# display summary and compare
import pandas as pd
Exercise_2_result = pd.DataFrame(Compare,
                      columns=['Nr_sim','K', 'N_p','N_lower', 'N_upper',
                               'N_intlen', 'CV_p','CV_lower', 'CV_upper',
                               'CV_intlen','CV_rho', 'diff_p', 'diff_intlen', 'times_intlen'])


Exercise_2_result

,Nr_sim,K,N_p,N_lower,N_upper,N_intlen,CV_p,CV_lower,CV_upper,CV_intlen,CV_rho,diff_p,diff_intlen,times_intlen
0,10000.0,80.0,23.421894,22.9072,23.9366,1.029357,23.237110,23.1014,23.3728,0.271324,0.974852,0.184785,0.758033,3.793827
1,100000.0,80.0,23.530067,23.3635,23.6966,0.333106,23.405544,23.3627,23.4484,0.085715,0.974610,0.124523,0.247391,3.886211
2,10000.0,150.0,1.457856,1.3061,1.6096,0.303420,1.637136,1.4847,1.7896,0.304961,0.590258,-0.179279,-0.001541,0.994947
3,100000.0,150.0,1.447885,1.3994,1.4963,0.096893,1.498206,1.4519,1.5445,0.092589,0.572336,-0.050322,0.004304,1.046486


### Comment: 
as we could see, when the correlation is big (CV_rho is near 1), the effect of variance reduction is obvious. The normal simulation almost have around 4.4 times confidence interval length compared to that of CVMC. However, when the correlation is small, the effect of variance reduction also decreases. 

## Exercise 3: Importance Sampling

In [181]:
# 3.1 Normal simulation
Ks = [2, 3.5, 5]
nr_sims = [1000000]

nr_cases = len(Ks) * len(nr_sims)

value_conf_interval = np.zeros((nr_cases, 6))

i = 0
for ki in range(len(Ks)): 
    
    K = Ks[ki]
    
    for si in range(len(nr_sims)): 
        
        nr_simi = nr_sims[si]
        X = np.random.randn(nr_simi, 1)
        
        biggerK_id = X > K
        
        p = np.sum(biggerK_id) / nr_simi
        
        var_n = nr_simi /(nr_simi - 1) * (p - p ** 2) 
        
        half_int = 1.96 * var_n ** 0.5 / nr_simi ** 0.5 
        conf_interval = [p - half_int,  p + half_int]
  
        value_conf_interval[i, 0] = nr_simi
        value_conf_interval[i, 1] = K
        value_conf_interval[i, 2] = p
        
        value_conf_interval[i, 3:5] = conf_interval
        value_conf_interval[i, 5] = half_int * 2
        
        i += 1
        
        
NMC = value_conf_interval        

In [182]:
# display normal Monte Carlo simulation result
result = pd.DataFrame(NMC,
                      columns=['Nr_sims', 'K', 'probability',
                               '95% conf_lower_bound', '95% conf_upper_bound','interval_length'])


result

,Nr_sims,K,probability,95% conf_lower_bound,95% conf_upper_bound,interval_length
0,1000000.0,2.0,0.022916,0.022623,0.023209,0.000587
1,1000000.0,3.5,0.000231,0.000201,0.000261,0.000060
2,1000000.0,5.0,0.000000,0.000000,0.000000,0.000000


In [183]:
# 3.2 Importance Sampling of simulation
Ks = [2, 3.5, 5]
nr_sims = [1000000]

nr_cases = len(Ks) * len(nr_sims)

value_conf_interval = np.zeros((nr_cases, 6))

i = 0
for ki in range(len(Ks)): 
    
    K = Ks[ki]
    
    for si in range(len(nr_sims)): 
        
        nr_simi = nr_sims[si]
        X = np.random.randn(nr_simi, 1)
        
        h = 1.1 * K
        biggerK_id = (X + h) > K
        
        XH = np.exp(- h * X - h ** 2 / 2) * biggerK_id
        p = np.mean(XH)
        
        
        var_n = nr_simi /(nr_simi - 1) * (np.mean(XH ** 2) - p ** 2) 
        
        half_int = 1.96 * var_n ** 0.5 / nr_simi ** 0.5 
        conf_interval = [p - half_int,  p + half_int]
  
        value_conf_interval[i, 0] = nr_simi
        value_conf_interval[i, 1] = K
        value_conf_interval[i, 2] = p
        
        value_conf_interval[i, 3:5] = conf_interval
        value_conf_interval[i, 5] = half_int * 2
        
        i += 1
        
ISMC = value_conf_interval      

In [184]:
# display Importance sampling Monte Carlo
import pandas as pd
result = pd.DataFrame(ISMC,
                      columns=['Nr_sims', 'K', 'probability',
                               '95% conf_lower_bound', '95% conf_upper_bound','interval_length'])


result

,Nr_sims,K,probability,95% conf_lower_bound,95% conf_upper_bound,interval_length
0,1000000.0,2.0,2.270866e-02,2.264168e-02,2.277564e-02,1.339619e-04
1,1000000.0,3.5,2.324889e-04,2.315765e-04,2.334012e-04,1.824651e-06
2,1000000.0,5.0,2.879761e-07,2.865716e-07,2.893807e-07,2.809124e-09


In [185]:
# Compare the result
abs_p_diff = NMC[:,2] - ISMC[:,2]
abs_conf_inter_diff = NMC[:,5] - ISMC[:,5] 
times_conf_inter_len = NMC[:,5] / ISMC[:,5] 
Compare = np.hstack((NMC, ISMC[:,2:], abs_p_diff[:,np.newaxis],
                     abs_conf_inter_diff[:,np.newaxis], times_conf_inter_len[:,np.newaxis]))


# display summary and compare
import pandas as pd
Exercise_3_result = pd.DataFrame(Compare,
                      columns=['Nr_sim', 'K', 'N_p','N_lower', 'N_upper',
                               'N_intlen', 'IS_p','IS_lower', 'IS_upper',
                               'IS_intlen','abs_diff_p', 'abs_diff_intlen', 'times_intlen'])


Exercise_3_result

,Nr_sim,K,N_p,N_lower,N_upper,N_intlen,IS_p,IS_lower,IS_upper,IS_intlen,abs_diff_p,abs_diff_intlen,times_intlen
0,1000000.0,2.0,0.022916,0.022623,0.023209,0.000587,2.270866e-02,2.264168e-02,2.277564e-02,1.339619e-04,2.073369e-04,4.526105e-04,4.378652
1,1000000.0,3.5,0.000231,0.000201,0.000261,0.000060,2.324889e-04,2.315765e-04,2.334012e-04,1.824651e-06,-1.488857e-06,5.774734e-05,32.648437
2,1000000.0,5.0,0.000000,0.000000,0.000000,0.000000,2.879761e-07,2.865716e-07,2.893807e-07,2.809124e-09,-2.879761e-07,-2.809124e-09,0.000000


## Comment: 

The effect of variance reduction is obvious. Here, we choose h = K. For K = 2 and 3.5, the confidence interval length is narrowed by around 4.4 and 32.6 times. For K = 5, the difference is negligiable due to the numerical precision, i.e. when K = 5, a variable following standard normal distrbution could hardly be over K. 